---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [138]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [126]:
#testing       
dates1 =  ["asdfa 23 Jun 2018 asdfa", "2 Dec 2017", "11 Jun 2018", 
        "01 January 2019", "10 mar. 15", "01 Jan 2007"] 

dates2 = ["asdfa 07/20/2017 asdf", "04/20/09", "4/20/09", "4/3/09"]

dates3= ['asdfa Mar 20, 2009 asdfa', 'March 20, 2009', 'Mar. 20, 2009', 'Mar 20 2009']

dates4 =  ["asdf Mar-20-2009 asdf", " sadf Feb-01-2015 asdf"]

dates5 = ["asdfa Mar 20th, 2009 asdfa", "Mar 21st, 2009", "Mar 22nd, 2009"]

dates6 = ["asdfa Feb 2009 asdfa", "Sep 2009", "Oct 2010"]

dates7 = ["asdfa 6/2008 asdfa", "12/2009"]

dates8 = ["asdf 2008 asdfasd", " as 2017 asad"]

dates = dates1+dates2+dates3+dates4+dates5+dates6+dates7+dates8

In [42]:
# 04/20/2009; 04/20/09; 4/20/09; 4/3/09
# Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
# 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
# Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
# Feb 2009; Sep 2009; Oct 2010
# 6/2008; 12/2009
# 2009; 2010

In [139]:
#where to check regex code
# https://regex101.com/

#import regex
import re

#import calendar
import calendar

short_month = {v: k for k,v in enumerate(calendar.month_abbr)}

# Import the datetime module 
from datetime import datetime 

In [70]:
i = dates1[0]
i

'23 Jun 2018'

In [193]:
#putting it all together

date_list = []

#find all dates of type month/day/year


for i in df:
    
    try:
        m = re.search('\d+(/)\d+(/)\d+', i)
        date = m.group(0)

        date = date.split("/")

        month = int(date[0])

        year = int(date[2][-2:])

        day = int(date[1])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
        
        continue
        
    except:
        pass

#find all dates of type day, name of month, year


    try:
    
        m = re.search('\d+ (.+) \d+', i)
        found1 = m.group(1)
        month = found1.lower()
        month = month.title()

        month = int(short_month[month[:3]])

        m = re.search('(\d+) .+ \d+', i, re.IGNORECASE)
        found2 = m.group(1)

        day = int(found2)

        m = re.search('\d+ .+ (\d+)', i, re.IGNORECASE)
        found3 = m.group(1)

        year = int(found3[-2:])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
    
        continue
    
    except:
        
        pass
    
#find all dates of type day, name of month, year part two
    
    try:
        
        m = re.search('(?<!\d)\d{4,7}(?!\d)', i)
        year = m.group(0)
        
        m = re.search(f'\d+ \w+ {year}', i)
        a = m.group(0)
        b = a.replace(f"{year}", "")
        day = [int(s) for s in b.split() if s.isdigit()][0]
        
        month = b.replace(f"{day}", "")
        month = month.replace(" ", "")
        month = month.lower()
        month = month.title()  
        month = int(short_month[month[:3]])
        
        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
        
        continue
        
    except:
        pass

    

#find all dates of type name month, day, year except month-day-year


    try:

        m = re.search('(.+) \d+', i)
        found1 = m.group(1)
        month = found1.lower()
        month = month.title()  
        month = int(short_month[month[:3]])

        m = re.search('(.+) \d+', i)
        found2 = m.group(1)
        day = found2
        m1 = re.search('.+ (\d+)', day)
        found3 = m1.group(1)
        day = int(found3)

        m = re.search('.+ (\d+)', i)
        found4 = m.group(1)
        year = int(found4[-2:])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
    
        continue
        
    except:
        pass
        
        
#find all dates of type name month-day-year

    try:

        m = re.search('\w+(-)\d+(-)\d+', i)
        date = m.group(0)
        date = date.split("-")

        month = date[0].lower()
        month = month.title()  
        month = int(short_month[month[:3]])

        day = int(date[1])

        year = int(date[2][-2:])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
    
        continue
    
    except:
        pass
    
#find all dates of type month-day-year

    try:

        m = re.search('\d+(-)\d+(-)\d+', i)
        date = m.group(0)
        date = date.split("-")

        month = date[0].lower()
        month = int(month) 

        day = int(date[1])

        year = int(date[2][-2:])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date) 

        continue

    except:
        pass


#find all dates with only name month and year

    try:

        m = re.search('.+ (\d+)', i)
        found1 = m.group(1)
        year = found1

        m = re.search(f'(.+) {year}', i)
        found2 = m.group(1)
        month = found2
        month = month.lower()
        month = month.title()  
        month = int(short_month[month[:3]])

        day = 1

        year = int(year[-2:])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)

        continue
        
    except:
        pass

#find all dates with month/year


    try:

        m = re.search('\d+(/)\d+', i)
        date = m.group(0)

        date = date.split("/")

        month = int(date[0])

        year = int(date[1][-2:])

        day = 1

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)

        continue
        
    except:
        pass
        
#find all dates with year

    try:

        word_list = i.split(" ")

        for a in word_list:

            if a.isalpha():
                continue

            else:
                year = a

        year = int(year[-2:])

        month = 1

        day = 1

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
    
        continue
    
    except:
        print(i)
        pass

s The patient is a 44 year old married Caucasian woman, unemployed Decorator, living with husband and caring for two young children, who is referred by Capitol Hill Hospital PCP, Dr. Heather Zubia, for urgent evaluation/treatment till first visit with Dr. Toney Winkler IN EIGHT WEEKS on 24 Jan 2001.

s The patient is a 49 year old married Canadian emigree, living with husband and two young adult children, with history of recurrent major depression with psychotic features and SI/SA, and alcoholism, requiring multiple hospitalizations for SI/SA and detox, who presents for bridge care s/p discharge from Placerville Sanatorium on 05 Feb 1992 and follow up outpatient mental health treatment at Anson General Hospital.

44 y/o MWF, unemployed Decorator, living with husband and caring for their two young children, referred by CHH PCP for urgent evaluation/treatment till first visit with Dr. Toney Winkler on 24 Jan 2011, presents with history of unclear rheumatologic illness and partially treat

In [196]:
i = "s The patient is a 44 year old married Caucasian woman, unemployed Decorator, living with husband and caring for two young children, who is referred by Capitol Hill Hospital PCP, Dr. Heather Zubia, for urgent evaluation/treatment till first visit with Dr. Toney Winkler IN EIGHT WEEKS on 24 Jan 2001."

In [197]:
m = re.search('(?<!\d)\d{4,7}(?!\d)', i)
year = m.group(0)
m = re.search(f'\d+ \w+ {year}', i)
a = m.group(0)

In [198]:
b = a.replace(f"{year}", "")
day = [int(s) for s in b.split() if s.isdigit()][0]
month = b.replace(f"{day}", "")
month = month.replace(" ", "")
month = month.lower()
month = month.title()  
month = int(short_month[month[:3]])
month

1

In [199]:
c = [int(s) for s in b.split() if s.isdigit()][0]
c

24

In [166]:
print(m)

<re.Match object; span=(295, 299), match='2001'>


In [163]:
date_list = []

m = re.search('\d+ (.+) \d+', i)
found1 = m.group(1)
month = found1.lower()
month = month.title()

month = int(short_month[month[:3]])

m = re.search('(\d+) .+ \d+', i, re.IGNORECASE)
found2 = m.group(1)

day = int(found2)

m = re.search('\d+ .+ (\d+)', i, re.IGNORECASE)
found3 = m.group(1)

year = int(found3[-2:])

date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
date = datetime.strptime(date, '%m/%d/%y')

date_list.append(date)


AttributeError: 'NoneType' object has no attribute 'group'

In [164]:
print(m)

None


In [194]:
len(df)

500

In [195]:
len(date_list)

337

In [157]:
sorted(date_list)

[datetime.datetime(1971, 4, 10, 0, 0),
 datetime.datetime(1971, 5, 18, 0, 0),
 datetime.datetime(1971, 7, 8, 0, 0),
 datetime.datetime(1971, 7, 11, 0, 0),
 datetime.datetime(1971, 9, 12, 0, 0),
 datetime.datetime(1972, 1, 13, 0, 0),
 datetime.datetime(1972, 1, 26, 0, 0),
 datetime.datetime(1972, 5, 6, 0, 0),
 datetime.datetime(1972, 5, 13, 0, 0),
 datetime.datetime(1972, 6, 10, 0, 0),
 datetime.datetime(1972, 6, 15, 0, 0),
 datetime.datetime(1972, 7, 20, 0, 0),
 datetime.datetime(1972, 10, 4, 0, 0),
 datetime.datetime(1972, 11, 30, 0, 0),
 datetime.datetime(1973, 2, 1, 0, 0),
 datetime.datetime(1973, 2, 14, 0, 0),
 datetime.datetime(1973, 3, 1, 0, 0),
 datetime.datetime(1973, 4, 1, 0, 0),
 datetime.datetime(1973, 6, 1, 0, 0),
 datetime.datetime(1973, 7, 1, 0, 0),
 datetime.datetime(1973, 10, 1, 0, 0),
 datetime.datetime(1973, 12, 1, 0, 0),
 datetime.datetime(1974, 2, 1, 0, 0),
 datetime.datetime(1974, 2, 24, 0, 0),
 datetime.datetime(1974, 3, 1, 0, 0),
 datetime.datetime(1974, 3, 5, 0,

In [14]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here